In [1]:
import sys
sys.path.append('..')

In [2]:
from core.dataset import *
from core.dprocessing import *
import seaborn as sns
import numpy as np
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
import seaborn as sns
from sklearn.metrics import classification_report

In [3]:
load_dir = 'D:/Data/hoffmanlab/featureselection/data/'
ligands = ['CpG', 'FLA', 'FSL', 'LPS', 'P3K', 'PIC', 'R84', 'TNF']
sheet_type = 'am'

In [13]:
dataset = Data(load_dir, ligands, sheet_type, merge=True, numpy=False)
labels = dataset.iloc[:, [984]]

In [5]:
features = ['integrals_pos', 'min_trough2peak', 'integrals', 'envelope', 'oscpower', 'fold_change', 'valley_amps']
time_steps = [98, None, 98, 25, None, 98, 15]

#### Training a model on all time series per feature for top 7 features
* integrals_pos
* min_trough2peak
* integrals
* envelope
* oscpower
* fold_change
* valley_amps

In [5]:
dataset['integrals_pos_1']

0       0
1       0
2       0
3       0
4       0
       ..
1597    0
1598    0
1599    0
1600    0
1601    0
Name: integrals_pos_1, Length: 14376, dtype: int64

In [6]:
feature_name = 'integrals_pos'

In [7]:
df = pd.DataFrame()
for i in range(1, 99):
    column = feature_name + '_' + str(i)
    column = dataset[column]
    df = pd.concat([df, column], axis=1)

In [8]:
# pass in list of feature names, pass in number of time steps for that corresponding feature

In [11]:
test2 = process(features, time_steps, dataset)

In [12]:
test2

,integrals_pos_1,integrals_pos_2,integrals_pos_3,integrals_pos_4,integrals_pos_5,integrals_pos_6,integrals_pos_7,integrals_pos_8,integrals_pos_9,integrals_pos_10,...,valley_amps_6,valley_amps_7,valley_amps_8,valley_amps_9,valley_amps_10,valley_amps_11,valley_amps_12,valley_amps_13,valley_amps_14,valley_amps_15
0,0,0.015096,0.032543,0.040461,0.041608,0.042510,0.051934,0.083181,0.139928,0.242935,...,5.988297,5.954039,5.959827,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.026792,0.105063,...,6.034079,6.104633,6.186130,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0.000000,0.006687,0.011071,0.011161,0.013339,0.022381,0.026237,0.056502,0.135171,...,5.529811,5.527464,5.293195,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0,0.000000,0.020693,0.172892,0.391335,0.603795,0.800702,0.971314,1.137790,1.286021,...,5.229372,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0,0.009170,0.019513,0.024005,0.031380,0.055451,0.105201,0.194234,0.325869,0.532726,...,5.821342,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1597,0,0.013909,0.013909,0.018214,0.048951,0.048951,0.048951,0.048951,0.048951,0.048951,...,5.790085,5.449458,5.181230,5.183025,NaN,NaN,NaN,NaN,NaN,NaN
1598,0,0.010901,0.010901,0.010901,0.010901,0.010901,0.060818,0.238731,0.305547,0.331205,...,5.318652,5.182635,5.213391,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1599,0,0.000000,0.000000,0.029628,0.040879,0.058672,0.076311,0.090650,0.120041,0.143540,...,5.388622,5.089885,5.195547,5.397384,NaN,NaN,NaN,NaN,NaN,NaN
1600,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.063195,0.189367,0.234028,0.234028,...,5.223900,5.427450,4.889019,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
model = xgb.XGBClassifier()

In [22]:
labels = dataset.iloc[:, [984]]

In [23]:
data, labels = test2.to_numpy(), labels.to_numpy()

In [24]:
labels = labels.reshape((-1, ))

In [25]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size = 0.1, random_state = 42)

In [26]:
model.fit(X_train, y_train)

[19:07:28] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [27]:
pred = model.predict(X_val)

In [28]:
pred

array([6, 6, 2, ..., 2, 0, 4])

In [29]:
y_val

array([1, 6, 2, ..., 6, 0, 0])

In [30]:
x = 0
for y_pred, y_true in zip(pred, y_val):
    if y_pred == y_true:
        x += 1
print(x / len(y_val))

0.6404728789986092


In [31]:
# almost same accuracy vs trained on all features (985), removed a lot of noise ig

In [42]:
cr = classification_report(y_val, pred, target_names=ligands)

In [44]:
print(cr)

              precision    recall  f1-score   support

         CpG       0.49      0.60      0.54       196
         FLA       0.49      0.45      0.47       143
         FSL       0.59      0.60      0.59       189
         LPS       0.58      0.72      0.64       149
         P3K       0.64      0.46      0.54       185
         PIC       0.82      0.71      0.76       165
         R84       0.78      0.83      0.80       249
         TNF       0.75      0.67      0.71       162

    accuracy                           0.64      1438
   macro avg       0.64      0.63      0.63      1438
weighted avg       0.65      0.64      0.64      1438



### given the entire array of the top features:
* parameters:
    * distinct features
    * upper bound (search in the upper x% of the array) for time steps for THOSE features

In [6]:
df = pd.read_csv('C:/Users/minha/Research/hoffmanlab/featuresel/data/untuned_model_feature_importance.csv')

In [7]:
feature_importance = df.values.tolist()

### trying 25% of top features for now:
* loop through all partitions and find the best?

In [10]:
data = partition_features(features, time_steps, feature_importance, 0.25, dataset)

In [16]:
data = data.to_numpy()
labels = labels.to_numpy()

In [17]:
labels = labels.reshape((-1, ))

In [18]:
X_train, X_val, y_train, y_val = train_test_split(data, labels, test_size=0.1, random_state=42)

In [21]:
model.fit(X_train, y_train)

C:\Users\minha\miniconda3\envs\peep\lib\site-packages\xgboost\sklearn.py:1224: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[21:11:41] WARNING: ..\src\learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=0, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=6, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=100, n_jobs=16,
              num_parallel_tree=1, objective='multi:softprob', predictor='auto',
              random_state=0, reg_alpha=0, reg_lambda=1, scale_pos_weight=None,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [24]:
pred = model.predict(X_val)

In [25]:
cr = classification_report(y_val, pred, target_names=ligands)

In [28]:
print(cr)

              precision    recall  f1-score   support

         CpG       0.46      0.54      0.49       196
         FLA       0.52      0.50      0.51       143
         FSL       0.57      0.58      0.58       189
         LPS       0.58      0.74      0.65       149
         P3K       0.66      0.51      0.57       185
         PIC       0.80      0.69      0.74       165
         R84       0.76      0.82      0.79       249
         TNF       0.76      0.64      0.70       162

    accuracy                           0.63      1438
   macro avg       0.64      0.63      0.63      1438
weighted avg       0.64      0.63      0.64      1438



### features trained on top 25% of timesteps for top 7 features yields 63% accuracy, 1% lower than model trained on 985 features

### make function to loop thru multiple bounds (partitions), find most optimal split (# of time steps for each feature)